Sujet: Test BNI 

Date: 26-27 décembre 2022.

Autheur: Jean-Christophe Busque

Table des matières

A) Importation des modules et des intrants

B) Nettoyage des données

C) Portfolio Return Series

D) Portfolio Index Series

E) Cumulative Outperformance

# Input et hypothèse

Je prend pour acquis que le code sera dans le même dossier que les intrants (Fichier Excel)

In [1]:
#Date de début de la normalisation des prix à 100$
start_date = "2020-03-05"

# A) Importation des modules et des intrants

## Importation des modules

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px

## Variables de base

In [3]:
#Path dans lequel les input et output du code se prennent/sauvegardent 
base_path = ''
base_path_out = base_path+"output_"


In [4]:
#input des données
in_data = pd.read_excel(base_path+'Technical Test - Portfolio Attribution.xlsm', sheet_name=['Benchmark Weights', 'SAA Weights', 'Manager Weights', 'Returns'], header=0)



In [5]:
#Séparation des données par feuilles
in_bmk_weights = in_data.get('Benchmark Weights')
in_saa_weights = in_data.get('SAA Weights')
in_manager_weights = in_data.get('Manager Weights')
in_ret = in_data.get('Returns')

display(in_manager_weights)

,Date,Equities,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Fixed Income,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,NaT,Canada,NaN,US,NaN,EAFE,NaN,EM,NaN,Canada Broad,NaN,Canada Corps,NaN
1,NaT,Fund A,Fund B,Fund C,Fund D,Fund E,Fund F,Fund G,Fund H,Fund I,Fund J,Fund K,Fund L
2,2022-09-15,0.011827,0.988173,0.382932,0.617068,0.640137,0.359863,0.5,0.5,0.608345,0.391655,0.238319,0.761681
3,2022-07-15,0.366969,0.633031,0.525906,0.474094,0.903722,0.096278,0.5,0.5,0.250429,0.749571,0.107232,0.892768
4,2021-09-15,0.761936,0.238064,0.361318,0.638682,0.262593,0.737407,1,0,0.104399,0.895601,0.442218,0.557782
5,2021-06-15,0.804688,0.195312,0.101631,0.898369,0.311963,0.688037,1,0,0.182061,0.817939,0.288275,0.711725
6,2020-07-02,0.544854,0.455146,0.271423,0.728577,1,0,1,0,0.678511,0.321489,0.546947,0.453053
7,2020-03-05,0.019857,0.980143,0.895803,0.104197,1,0,1,0,0.264452,0.735548,0.17464,0.82536


# B) Nettoyage des données

## Returns

In [6]:
display(in_ret.head(3))

,Date,Index Total Returns (CAD),Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Fund Total Returns (CAD),Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,NaT,US equities,Canada equities,EAFE equities,EM equities,Canada Broad FI,Canada Corps FI,Fund A,Fund B,Fund C,Fund D,Fund E,Fund F,Fund G,Fund H,Fund I,Fund J,Fund K,Fund L
1,2022-10-31,-0.005316,-0.001917,0.002864,0.005212,-0.002097,-0.001536,-0.011583,-0.003497,-0.005087,-0.007031,-0.007353,0.002165,-0.00289,-0.001899,-0.002141,-0.001595,-0.002086,-0.004728
2,2022-10-28,0.031842,0.006303,-0.000377,-0.009328,-0.000627,-0.000799,0.015324,0.008818,0.02917,0.020734,0.01169,0.000236,0.002899,0.004771,-0.00107,-0.000634,0,-0.001181


In [7]:
#Renaming columns
ret = in_ret.rename(columns={
'Index Total Returns (CAD)': 'IDX - US equities',
'Unnamed: 2': 'IDX - Canada equities',
'Unnamed: 3': 'IDX - EAFE equities',
'Unnamed: 4': 'IDX - EM equities',
'Unnamed: 5': 'IDX - Canada Broad FI',
'Unnamed: 6': 'IDX - Canada Corps FI',
'Fund Total Returns (CAD)': 'Fund A - Canada equities',
'Unnamed: 8': 'Fund B - Canada equities',
'Unnamed: 9': 'Fund C - US equities',
'Unnamed: 10': 'Fund D - US equities',
'Unnamed: 11': 'Fund E - EAFE equities',
'Unnamed: 12': 'Fund F - EAFE equities',
'Unnamed: 13': 'Fund G - EM equities',
'Unnamed: 14': 'Fund H - EM equities',
'Unnamed: 15': 'Fund I - Canada Broad FI',
'Unnamed: 16': 'Fund J - Canada Broad FI',
'Unnamed: 17': 'Fund K - Canada Corps FI',
'Unnamed: 18': 'Fund L - Canada Corps FI'})

In [8]:
#Drop row 0 and set index as Date
ret = ret.drop(labels=0, axis=0)
ret = ret.set_index("Date")

In [9]:
#Fill na with zéro
ret = ret.fillna(0)

In [10]:
#Sort index by date (past to now)
ret = ret.sort_index(ascending=True)

In [11]:
display(ret.head())

,IDX - US equities,IDX - Canada equities,IDX - EAFE equities,IDX - EM equities,IDX - Canada Broad FI,IDX - Canada Corps FI,Fund A - Canada equities,Fund B - Canada equities,Fund C - US equities,Fund D - US equities,Fund E - EAFE equities,Fund F - EAFE equities,Fund G - EM equities,Fund H - EM equities,Fund I - Canada Broad FI,Fund J - Canada Broad FI,Fund K - Canada Corps FI,Fund L - Canada Corps FI
Date,,,,,,,,,,,,,,,,,,
2020-03-06,-0.014991,-0.022876,-0.021158,-0.024204,0.010489,0.004190,-0.017890,-0.022817,-0.016579,-0.017783,-0.014418,0.0,-0.021544,-0.004859,0.008772,0.009372,0.003487,-0.001970
2020-03-09,-0.069109,-0.102410,-0.050206,-0.056530,0.008591,-0.003267,-0.070528,-0.095431,-0.059450,-0.051652,-0.054336,0.0,-0.042202,-0.033203,0.007826,0.009257,-0.004344,-0.007897
2020-03-10,0.068525,0.030798,0.008682,0.036086,-0.009319,-0.006491,0.032663,0.019080,0.055660,0.052256,0.026519,0.0,0.044061,0.017172,-0.008628,-0.007594,-0.006108,-0.016915
2020-03-11,-0.050423,-0.045924,-0.019074,-0.020115,-0.010823,-0.008303,-0.031630,-0.039648,-0.050045,-0.043237,-0.043057,0.0,-0.039450,-0.027805,-0.009574,-0.010419,-0.008780,-0.003036
2020-03-12,-0.086818,-0.123306,-0.095560,-0.058681,-0.011579,-0.013525,-0.081910,-0.105505,-0.085607,-0.085370,-0.102362,0.0,-0.082139,-0.055158,-0.010545,-0.011452,-0.013286,-0.016244


In [12]:
#Extraction of all dates
all_dates = ret.index.to_list()


## Benchmark

In [13]:
display(in_bmk_weights)

,Equities,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Fixed Income
0,Date,US,Canada,EAFE,EM,Canada Broad
1,2020-03-05 00:00:00,0.21,0.21,0.12,0.06,0.4


In [14]:
#Renaming columns
bmk_weights = in_bmk_weights.rename(columns={
'Equities': 'Date',
'Unnamed: 1': 'IDX - US equities',
'Unnamed: 2': 'IDX - Canada equities',
'Unnamed: 3': 'IDX - EAFE equities',
'Unnamed: 4': 'IDX - EM equities', 
'Fixed Income': 'IDX - Canada Broad FI'})


In [15]:
#Drop row 0 and set index as Date
bmk_weights = bmk_weights.drop(labels=0, axis=0)
bmk_weights = bmk_weights.set_index("Date")

In [16]:
display(bmk_weights)

,IDX - US equities,IDX - Canada equities,IDX - EAFE equities,IDX - EM equities,IDX - Canada Broad FI
Date,,,,,
2020-03-05,0.21,0.21,0.12,0.06,0.4


## SAA

In [17]:
display(in_saa_weights)

,Date,Equities,Unnamed: 2,Unnamed: 3,Unnamed: 4,Fixed Income,Unnamed: 6
0,NaT,Canada,US,EAFE,EM,Canada Broad,Canada Corps
1,2022-07-15,0.22,0.22,0.13,0.07,0.3,0.06
2,2021-06-15,0.23,0.19,0.12,0.06,0.35,0.05
3,2020-03-05,0.21,0.21,0.12,0.06,0.35,0.05


In [18]:
#Renaming columns
saa_weights = in_saa_weights.rename(columns={
'Equities': 'IDX - Canada equities',
'Unnamed: 2': 'IDX - US equities',
'Unnamed: 3': 'IDX - EAFE equities',
'Unnamed: 4': 'IDX - EM equities',
'Fixed Income': 'IDX - Canada Broad FI',
'Unnamed: 6': 'IDX - Canada Corps FI'})

In [19]:
#Drop row 0 and set index as Date
saa_weights = saa_weights.drop(labels=0, axis=0)
saa_weights = saa_weights.set_index("Date")

In [20]:
display(saa_weights)

,IDX - Canada equities,IDX - US equities,IDX - EAFE equities,IDX - EM equities,IDX - Canada Broad FI,IDX - Canada Corps FI
Date,,,,,,
2022-07-15,0.22,0.22,0.13,0.07,0.3,0.06
2021-06-15,0.23,0.19,0.12,0.06,0.35,0.05
2020-03-05,0.21,0.21,0.12,0.06,0.35,0.05


## Manager

In [21]:
display(in_manager_weights)

,Date,Equities,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Fixed Income,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,NaT,Canada,NaN,US,NaN,EAFE,NaN,EM,NaN,Canada Broad,NaN,Canada Corps,NaN
1,NaT,Fund A,Fund B,Fund C,Fund D,Fund E,Fund F,Fund G,Fund H,Fund I,Fund J,Fund K,Fund L
2,2022-09-15,0.011827,0.988173,0.382932,0.617068,0.640137,0.359863,0.5,0.5,0.608345,0.391655,0.238319,0.761681
3,2022-07-15,0.366969,0.633031,0.525906,0.474094,0.903722,0.096278,0.5,0.5,0.250429,0.749571,0.107232,0.892768
4,2021-09-15,0.761936,0.238064,0.361318,0.638682,0.262593,0.737407,1,0,0.104399,0.895601,0.442218,0.557782
5,2021-06-15,0.804688,0.195312,0.101631,0.898369,0.311963,0.688037,1,0,0.182061,0.817939,0.288275,0.711725
6,2020-07-02,0.544854,0.455146,0.271423,0.728577,1,0,1,0,0.678511,0.321489,0.546947,0.453053
7,2020-03-05,0.019857,0.980143,0.895803,0.104197,1,0,1,0,0.264452,0.735548,0.17464,0.82536


In [22]:
#Renaming columns
manager_weights = in_manager_weights.rename(columns={
'Equities': 'Fund A - Canada equities',
'Unnamed: 2': 'Fund B - Canada equities',
'Unnamed: 3': 'Fund C - US equities',
'Unnamed: 4': 'Fund D - US equities',
'Unnamed: 5': 'Fund E - EAFE equities',
'Unnamed: 6': 'Fund F - EAFE equities',
'Unnamed: 7': 'Fund G - EM equities',
'Unnamed: 8': 'Fund H - EM equities',
'Fixed Income': 'Fund I - Canada Broad FI',
'Unnamed: 10': 'Fund J - Canada Broad FI',
'Unnamed: 11': 'Fund K - Canada Corps FI',
'Unnamed: 12': 'Fund L - Canada Corps FI'})

In [23]:
#Drop row 0 & 1 and set index as Date
manager_weights = manager_weights.drop(labels=[0,1], axis=0)
manager_weights = manager_weights.set_index("Date")

In [24]:
display(manager_weights)

,Fund A - Canada equities,Fund B - Canada equities,Fund C - US equities,Fund D - US equities,Fund E - EAFE equities,Fund F - EAFE equities,Fund G - EM equities,Fund H - EM equities,Fund I - Canada Broad FI,Fund J - Canada Broad FI,Fund K - Canada Corps FI,Fund L - Canada Corps FI
Date,,,,,,,,,,,,
2022-09-15,0.011827,0.988173,0.382932,0.617068,0.640137,0.359863,0.5,0.5,0.608345,0.391655,0.238319,0.761681
2022-07-15,0.366969,0.633031,0.525906,0.474094,0.903722,0.096278,0.5,0.5,0.250429,0.749571,0.107232,0.892768
2021-09-15,0.761936,0.238064,0.361318,0.638682,0.262593,0.737407,1,0,0.104399,0.895601,0.442218,0.557782
2021-06-15,0.804688,0.195312,0.101631,0.898369,0.311963,0.688037,1,0,0.182061,0.817939,0.288275,0.711725
2020-07-02,0.544854,0.455146,0.271423,0.728577,1,0,1,0,0.678511,0.321489,0.546947,0.453053
2020-03-05,0.019857,0.980143,0.895803,0.104197,1,0,1,0,0.264452,0.735548,0.17464,0.82536


In [25]:
#Suppression des intrants puisque ceux-si sont traités. Libération de mémoire
del in_data
del in_bmk_weights
del in_manager_weights
del in_saa_weights
del in_ret

# C) Portfolio Return Series

## Benchmark portfolio (daily rebalanced)


In [26]:
#Fitler ret with bmk_assets
bmk_assets = bmk_weights.columns.values.tolist()
bmk_ret = ret[bmk_assets]

In [27]:
#Empty table with all dates where we have ret, to be filled 
temp_bmk = bmk_weights.reindex(all_dates, fill_value=None)

#Concat ce que nous avons + ce que nous voulons remplir
bmk_weights = pd.concat([bmk_weights, temp_bmk])

#Suppression des doublons pour conserver les poids des managers à certaines dates
bmk_weights = bmk_weights[~bmk_weights.index.duplicated(keep='first')]

#Forward Fill all missing dates in bmk_weights
bmk_weights = bmk_weights.sort_index(ascending=True)
bmk_weights = bmk_weights.ffill()

display(bmk_weights)

,IDX - US equities,IDX - Canada equities,IDX - EAFE equities,IDX - EM equities,IDX - Canada Broad FI
Date,,,,,
2020-03-05,0.21,0.21,0.12,0.06,0.4
2020-03-06,0.21,0.21,0.12,0.06,0.4
2020-03-09,0.21,0.21,0.12,0.06,0.4
2020-03-10,0.21,0.21,0.12,0.06,0.4
2020-03-11,0.21,0.21,0.12,0.06,0.4
...,...,...,...,...,...
2022-10-25,0.21,0.21,0.12,0.06,0.4
2022-10-26,0.21,0.21,0.12,0.06,0.4
2022-10-27,0.21,0.21,0.12,0.06,0.4


In [28]:
#Rendemement du portefeuilles par type d'actifs et total
bmk_ret_series = bmk_ret * bmk_weights
bmk_ret_series["Total return bmk"] = bmk_ret_series.sum(axis=1)

#Gardons la dernière cols
bmk_ret_series = bmk_ret_series[["Total return bmk"]]

display(bmk_ret_series)

,Total return bmk
Date,
2020-03-05,0.000000
2020-03-06,-0.007748
2020-03-09,-0.041999
2020-03-10,0.020337
2020-03-11,-0.028058
...,...
2022-10-25,0.007113
2022-10-26,0.006795
2022-10-27,0.001281


In [29]:
#Rendemement du portefeuilles par type d'actifs et total
bmk_ret_series = bmk_ret * bmk_weights
bmk_ret_series["Total return bmk"] = bmk_ret_series.sum(axis=1)

#Gardons la dernière cols
bmk_ret_series = bmk_ret_series[["Total return bmk"]]

display(bmk_ret_series)

,Total return bmk
Date,
2020-03-05,0.000000
2020-03-06,-0.007748
2020-03-09,-0.041999
2020-03-10,0.020337
2020-03-11,-0.028058
...,...
2022-10-25,0.007113
2022-10-26,0.006795
2022-10-27,0.001281


## SAA portfolio (daily rebalanced)

In [30]:
#Fitler ret with saa_assets
saa_assets = saa_weights.columns.values.tolist()
saa_ret = ret[saa_assets]

In [31]:
#Save for maanger weight with drift
saa_w_drift = saa_weights

In [32]:
#Fitler ret with saa_assets
saa_assets = saa_weights.columns.values.tolist()
saa_ret = ret[saa_assets]

In [33]:
#Empty table with all dates where we have ret, to be filled 
temp_saa = saa_weights.reindex(all_dates, fill_value=None)

#Concat ce que nous avons + ce que nous voulons remplir
saa_weights = pd.concat([saa_weights, temp_saa])

#Suppression des doublons pour conserver les poids des managers à certaines dates
saa_weights = saa_weights[~saa_weights.index.duplicated(keep='first')]

#Forward Fill all missing dates in saa_weights
saa_weights = saa_weights.sort_index(ascending=True)
saa_weights = saa_weights.ffill()

display(saa_weights)

,IDX - Canada equities,IDX - US equities,IDX - EAFE equities,IDX - EM equities,IDX - Canada Broad FI,IDX - Canada Corps FI
Date,,,,,,
2020-03-05,0.21,0.21,0.12,0.06,0.35,0.05
2020-03-06,0.21,0.21,0.12,0.06,0.35,0.05
2020-03-09,0.21,0.21,0.12,0.06,0.35,0.05
2020-03-10,0.21,0.21,0.12,0.06,0.35,0.05
2020-03-11,0.21,0.21,0.12,0.06,0.35,0.05
...,...,...,...,...,...,...
2022-10-25,0.22,0.22,0.13,0.07,0.30,0.06
2022-10-26,0.22,0.22,0.13,0.07,0.30,0.06
2022-10-27,0.22,0.22,0.13,0.07,0.30,0.06


In [34]:
#Sorting columns by names for multiplication
saa_weights = saa_weights.reindex(sorted(saa_weights.columns), axis=1)
saa_ret = saa_ret.reindex(sorted(saa_ret.columns), axis=1)

display(saa_weights, saa_ret)


,IDX - Canada Broad FI,IDX - Canada Corps FI,IDX - Canada equities,IDX - EAFE equities,IDX - EM equities,IDX - US equities
Date,,,,,,
2020-03-05,0.35,0.05,0.21,0.12,0.06,0.21
2020-03-06,0.35,0.05,0.21,0.12,0.06,0.21
2020-03-09,0.35,0.05,0.21,0.12,0.06,0.21
2020-03-10,0.35,0.05,0.21,0.12,0.06,0.21
2020-03-11,0.35,0.05,0.21,0.12,0.06,0.21
...,...,...,...,...,...,...
2022-10-25,0.30,0.06,0.22,0.13,0.07,0.22
2022-10-26,0.30,0.06,0.22,0.13,0.07,0.22
2022-10-27,0.30,0.06,0.22,0.13,0.07,0.22


,IDX - Canada Broad FI,IDX - Canada Corps FI,IDX - Canada equities,IDX - EAFE equities,IDX - EM equities,IDX - US equities
Date,,,,,,
2020-03-06,0.010489,0.004190,-0.022876,-0.021158,-0.024204,-0.014991
2020-03-09,0.008591,-0.003267,-0.102410,-0.050206,-0.056530,-0.069109
2020-03-10,-0.009319,-0.006491,0.030798,0.008682,0.036086,0.068525
2020-03-11,-0.010823,-0.008303,-0.045924,-0.019074,-0.020115,-0.050423
2020-03-12,-0.011579,-0.013525,-0.123306,-0.095560,-0.058681,-0.086818
...,...,...,...,...,...,...
2022-10-25,0.005312,0.003460,0.010091,0.011964,-0.005647,0.008439
2022-10-26,0.014302,0.011309,0.009570,0.010602,0.004253,-0.011726
2022-10-27,0.007244,0.005439,0.003753,-0.006788,0.005862,-0.009248


In [35]:
#Rendemement du portefeuilles par type d'actifs et total
saa_ret_series = saa_ret * saa_weights
saa_ret_series["Total return saa"] = saa_ret_series.sum(axis=1)

#Gardons la dernière cols
saa_ret_series = saa_ret_series[["Total return saa"]]

display(saa_ret_series)

,Total return saa
Date,
2020-03-05,0.000000
2020-03-06,-0.008063
2020-03-09,-0.042592
2020-03-10,0.020479
2020-03-11,-0.027932
...,...
2022-10-25,0.007038
2022-10-26,0.006171
2022-10-27,0.000819


## Manager portfolio (daily rebalanced on both levels)

In [36]:
#Fitler ret with manager_assets
manager_assets = manager_weights.columns.values.tolist()
manager_ret = ret[manager_assets]

In [37]:
#Save for manager with drift
manager_w_drift = manager_weights

In [38]:
#Fitler ret with saa_assets
manager_assets = manager_weights.columns.values.tolist()
manager_ret = ret[manager_assets]

In [39]:
#Save for manager with drift
ret_w_drift = manager_ret

In [40]:
#Empty table with all dates where we have ret, to be filled 
temp_manager = manager_weights.reindex(all_dates, fill_value=None)

#Concat ce que nous avons + ce que nous voulons remplir
manager_weights = pd.concat([manager_weights, temp_manager])

#Suppression des doublons pour conserver les poids des managers à certaines dates
manager_weights = manager_weights[~manager_weights.index.duplicated(keep='first')]

#Forward Fill all missing dates in manager_weights
manager_weights = manager_weights.sort_index(ascending=True)
manager_weights = manager_weights.ffill()

display(manager_weights)

,Fund A - Canada equities,Fund B - Canada equities,Fund C - US equities,Fund D - US equities,Fund E - EAFE equities,Fund F - EAFE equities,Fund G - EM equities,Fund H - EM equities,Fund I - Canada Broad FI,Fund J - Canada Broad FI,Fund K - Canada Corps FI,Fund L - Canada Corps FI
Date,,,,,,,,,,,,
2020-03-05,0.019857,0.980143,0.895803,0.104197,1.000000,0.000000,1.0,0.0,0.264452,0.735548,0.174640,0.825360
2020-03-06,0.019857,0.980143,0.895803,0.104197,1.000000,0.000000,1.0,0.0,0.264452,0.735548,0.174640,0.825360
2020-03-09,0.019857,0.980143,0.895803,0.104197,1.000000,0.000000,1.0,0.0,0.264452,0.735548,0.174640,0.825360
2020-03-10,0.019857,0.980143,0.895803,0.104197,1.000000,0.000000,1.0,0.0,0.264452,0.735548,0.174640,0.825360
2020-03-11,0.019857,0.980143,0.895803,0.104197,1.000000,0.000000,1.0,0.0,0.264452,0.735548,0.174640,0.825360
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-25,0.011827,0.988173,0.382932,0.617068,0.640137,0.359863,0.5,0.5,0.608345,0.391655,0.238319,0.761681
2022-10-26,0.011827,0.988173,0.382932,0.617068,0.640137,0.359863,0.5,0.5,0.608345,0.391655,0.238319,0.761681
2022-10-27,0.011827,0.988173,0.382932,0.617068,0.640137,0.359863,0.5,0.5,0.608345,0.391655,0.238319,0.761681


In [41]:
#Sorting columns by names for multiplication
manager_weights = manager_weights.reindex(sorted(manager_weights.columns), axis=1)
manager_ret = manager_ret.reindex(sorted(manager_ret.columns), axis=1)

display(manager_weights, manager_ret)

,Fund A - Canada equities,Fund B - Canada equities,Fund C - US equities,Fund D - US equities,Fund E - EAFE equities,Fund F - EAFE equities,Fund G - EM equities,Fund H - EM equities,Fund I - Canada Broad FI,Fund J - Canada Broad FI,Fund K - Canada Corps FI,Fund L - Canada Corps FI
Date,,,,,,,,,,,,
2020-03-05,0.019857,0.980143,0.895803,0.104197,1.000000,0.000000,1.0,0.0,0.264452,0.735548,0.174640,0.825360
2020-03-06,0.019857,0.980143,0.895803,0.104197,1.000000,0.000000,1.0,0.0,0.264452,0.735548,0.174640,0.825360
2020-03-09,0.019857,0.980143,0.895803,0.104197,1.000000,0.000000,1.0,0.0,0.264452,0.735548,0.174640,0.825360
2020-03-10,0.019857,0.980143,0.895803,0.104197,1.000000,0.000000,1.0,0.0,0.264452,0.735548,0.174640,0.825360
2020-03-11,0.019857,0.980143,0.895803,0.104197,1.000000,0.000000,1.0,0.0,0.264452,0.735548,0.174640,0.825360
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-25,0.011827,0.988173,0.382932,0.617068,0.640137,0.359863,0.5,0.5,0.608345,0.391655,0.238319,0.761681
2022-10-26,0.011827,0.988173,0.382932,0.617068,0.640137,0.359863,0.5,0.5,0.608345,0.391655,0.238319,0.761681
2022-10-27,0.011827,0.988173,0.382932,0.617068,0.640137,0.359863,0.5,0.5,0.608345,0.391655,0.238319,0.761681


,Fund A - Canada equities,Fund B - Canada equities,Fund C - US equities,Fund D - US equities,Fund E - EAFE equities,Fund F - EAFE equities,Fund G - EM equities,Fund H - EM equities,Fund I - Canada Broad FI,Fund J - Canada Broad FI,Fund K - Canada Corps FI,Fund L - Canada Corps FI
Date,,,,,,,,,,,,
2020-03-06,-0.017890,-0.022817,-0.016579,-0.017783,-0.014418,0.000000,-0.021544,-0.004859,0.008772,0.009372,0.003487,-0.001970
2020-03-09,-0.070528,-0.095431,-0.059450,-0.051652,-0.054336,0.000000,-0.042202,-0.033203,0.007826,0.009257,-0.004344,-0.007897
2020-03-10,0.032663,0.019080,0.055660,0.052256,0.026519,0.000000,0.044061,0.017172,-0.008628,-0.007594,-0.006108,-0.016915
2020-03-11,-0.031630,-0.039648,-0.050045,-0.043237,-0.043057,0.000000,-0.039450,-0.027805,-0.009574,-0.010419,-0.008780,-0.003036
2020-03-12,-0.081910,-0.105505,-0.085607,-0.085370,-0.102362,0.000000,-0.082139,-0.055158,-0.010545,-0.011452,-0.013286,-0.016244
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-25,0.014866,0.009901,0.010424,0.010156,0.013919,0.013632,-0.000971,0.005814,0.004920,0.005002,0.004220,0.008729
2022-10-26,0.005359,0.008913,-0.009580,-0.017788,0.004224,0.014673,0.005831,0.006744,0.014208,0.013419,0.010593,0.003571
2022-10-27,-0.002843,0.001767,-0.005208,-0.012598,-0.010515,-0.011290,0.000000,0.002871,0.007543,0.007216,0.005241,0.004745


In [42]:
#Compréhension
#Le manager décide d'investir entre deux fonds pour chaque géographie
#Lesquels étant pondérés par SAA
#nous devons donc rebalancer au niveau manager puis ensuite SAA

In [43]:
#Rendemement du portefeuilles par type d'actifs et total au niveau du manager
manager_ret_series = manager_ret * manager_weights

#Sommes des rendements par choix du manager
manager_ret_series["IDX - Canada equities"] = manager_ret_series.filter(like="Canada equities").sum(axis=1)
manager_ret_series["IDX - US equities"] = manager_ret_series.filter(like="US equities").sum(axis=1)
manager_ret_series["IDX - EAFE equities"] = manager_ret_series.filter(like="EAFE equities").sum(axis=1)
manager_ret_series["IDX - EM equities"] = manager_ret_series.filter(like="EM equities").sum(axis=1)
manager_ret_series["IDX - Canada Broad FI"] = manager_ret_series.filter(like="Canada Broad FI").sum(axis=1)
manager_ret_series["IDX - Canada Corps FI"] = manager_ret_series.filter(like="Canada Corps FI").sum(axis=1)

#Conservons les colonnes pertinentes
manager_ret_series = manager_ret_series.drop(manager_ret_series.filter(regex='Fund').columns, axis=1)


display(manager_ret_series)

,IDX - Canada equities,IDX - US equities,IDX - EAFE equities,IDX - EM equities,IDX - Canada Broad FI,IDX - Canada Corps FI
Date,,,,,,
2020-03-05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2020-03-06,-0.022720,-0.016705,-0.014418,-0.021544,0.009214,-0.001017
2020-03-09,-0.094937,-0.058637,-0.054336,-0.042202,0.008879,-0.007277
2020-03-10,0.019349,0.055306,0.026519,0.044061,-0.007867,-0.015028
2020-03-11,-0.039488,-0.049335,-0.043057,-0.039450,-0.010195,-0.004039
...,...,...,...,...,...,...
2022-10-25,0.009960,0.010259,0.013815,0.002422,0.004952,0.007655
2022-10-26,0.008871,-0.014645,0.007984,0.006287,0.013899,0.005245
2022-10-27,0.001712,-0.009769,-0.010794,0.001435,0.007415,0.004863


In [44]:
#Rendemement du portefeuilles par type d'actifs et total au niveau du manager pondérée par les choix SAA

#Sorting columns by names for multiplication
saa_weights = saa_weights.reindex(sorted(saa_weights.columns), axis=1)
manager_ret_series = manager_ret_series.reindex(sorted(manager_ret_series.columns), axis=1)

display(manager_ret_series, saa_weights)

,IDX - Canada Broad FI,IDX - Canada Corps FI,IDX - Canada equities,IDX - EAFE equities,IDX - EM equities,IDX - US equities
Date,,,,,,
2020-03-05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2020-03-06,0.009214,-0.001017,-0.022720,-0.014418,-0.021544,-0.016705
2020-03-09,0.008879,-0.007277,-0.094937,-0.054336,-0.042202,-0.058637
2020-03-10,-0.007867,-0.015028,0.019349,0.026519,0.044061,0.055306
2020-03-11,-0.010195,-0.004039,-0.039488,-0.043057,-0.039450,-0.049335
...,...,...,...,...,...,...
2022-10-25,0.004952,0.007655,0.009960,0.013815,0.002422,0.010259
2022-10-26,0.013899,0.005245,0.008871,0.007984,0.006287,-0.014645
2022-10-27,0.007415,0.004863,0.001712,-0.010794,0.001435,-0.009769


,IDX - Canada Broad FI,IDX - Canada Corps FI,IDX - Canada equities,IDX - EAFE equities,IDX - EM equities,IDX - US equities
Date,,,,,,
2020-03-05,0.35,0.05,0.21,0.12,0.06,0.21
2020-03-06,0.35,0.05,0.21,0.12,0.06,0.21
2020-03-09,0.35,0.05,0.21,0.12,0.06,0.21
2020-03-10,0.35,0.05,0.21,0.12,0.06,0.21
2020-03-11,0.35,0.05,0.21,0.12,0.06,0.21
...,...,...,...,...,...,...
2022-10-25,0.30,0.06,0.22,0.13,0.07,0.22
2022-10-26,0.30,0.06,0.22,0.13,0.07,0.22
2022-10-27,0.30,0.06,0.22,0.13,0.07,0.22


In [45]:
#Rendemement du portefeuilles du manager pondéré par les poids saa
manager_saa_ret_series = manager_ret_series * saa_weights
manager_saa_ret_series["Total return manager-saa"] = manager_saa_ret_series.sum(axis=1)

#Gardons la dernière cols
manager_saa_ret_series = manager_saa_ret_series[["Total return manager-saa"]]

display(manager_saa_ret_series)

,Total return manager-saa
Date,
2020-03-05,0.000000
2020-03-06,-0.008128
2020-03-09,-0.038559
2020-03-10,0.017999
2020-03-11,-0.029957
...,...
2022-10-25,0.008358
2022-10-26,0.004692
2022-10-27,-0.000559


## Manager portfolio (rebalanced on target dates only, drift between)

In [46]:
#Optique: Laisser les % évoluer dans le temps et les replacer selon les deux niveaux aux dates précises
#Réalisation en terme de $

Voici ce que j'aurais aimé effectué

Setup une valeur de portefeuille de 100$ au 2020-03-05 et la distribuer en cascade. La première distribution s'effectue selon SAA (Macro/Type d'investissement) puis ensuite au niveau du Manager (Funds).

Une fois que nous avons les $ investis dans chaque fonds, nous laissons croître ces montants selon le rendement des Funds jours par jours jusqu'à une date de rebalancement.

Lorsque nous atteignons une date de rebalancement Manager (Funds), nous calculons les montants investis (en date du closing d'hier) par géographie et type d'ivestissement puis nous venons réatribuer cette somme à travers les Funds selon les nouvelles pondération du gestionnaire.

Lorsque nous atteignons une date de rebalancement SAA (Macro), nous avons aussi un rebalancement Manager (Funds). Donc, nous calculons la valeur total du portefeuille (en date du closing d'hier) pour venir atrribuer une première fois selon SAA (Macro) et ensuite une deuxième répartition selon le Manager (Funds).

Des approches par dictionnaire ou classe seraient intéressantes.


## Visual

In [47]:
#Regroupons les données
all_ret_series = pd.concat([bmk_ret_series, saa_ret_series, manager_saa_ret_series], axis= 1)


In [48]:
fig = px.line(all_ret_series, x=all_ret_series.index, y=all_ret_series.columns,

              title='Portfolio return series')
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")
fig.show()


## Export

In [49]:
all_ret_series.to_csv(base_path_out+"Portfolio Return Series.csv")
fig.write_html(base_path_out+"Portfolio Return Series.html")


# D) Portfolio Index Series

## Function

In [50]:
def index(start_date):

    norm_ret_series = np.log(1+all_ret_series)
    norm_ret_series = norm_ret_series.loc[start_date:]

    #Set bgn price = 100
    norm_ret_series.iloc[:1] = 0

    #Normalize
    ##Bmk
    norm_ret_series["bmk cumsum"] = norm_ret_series["Total return bmk"].cumsum()  
    norm_ret_series["bmk exp"] = np.exp(norm_ret_series["bmk cumsum"])
    norm_ret_series["bmk price"] = 100*(norm_ret_series["bmk exp"])

    ##saa
    norm_ret_series["saa cumsum"] = norm_ret_series["Total return saa"].cumsum()  
    norm_ret_series["saa exp"] = np.exp(norm_ret_series["saa cumsum"])
    norm_ret_series["saa price"] = 100*(norm_ret_series["saa exp"])

    ##manager-saa
    norm_ret_series["manager-saa cumsum"] = norm_ret_series["Total return manager-saa"].cumsum()  
    norm_ret_series["manager-saa exp"] = np.exp(norm_ret_series["manager-saa cumsum"])
    norm_ret_series["manager-saa price"] = 100*(norm_ret_series["manager-saa exp"])

    #Keep relevant columns
    norm_ret_series = norm_ret_series[["bmk price", "saa price", "manager-saa price"]]
    
    return(norm_ret_series)


## Visual

In [51]:
fig = px.line(index(start_date), x=index(start_date).index, y=index(start_date).columns,

              title='Portfolio Index Series')

fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")
fig.show()

## Export

In [52]:
all_ret_series.to_csv(base_path_out+"Portfolio Index Series.csv")
fig.write_html(base_path_out+"Portfolio Index Series.html")

# E) Cumulative Outperformance

In [53]:
#Df creation
cumulative_outperformance = all_ret_series

cumulative_outperformance["saa minus bmk"] = (cumulative_outperformance["Total return saa"] - cumulative_outperformance["Total return bmk"]).cumsum()
cumulative_outperformance["manager-saa minus saa"] = (cumulative_outperformance["Total return manager-saa"] - cumulative_outperformance["Total return saa"]).cumsum()
#cumulative_outperformance["manager with drift minus manager"] = (cumulative_outperformance["Total return manager with drift"] - cumulative_outperformance["Total return manager"]).cumsum()


#Drop les columns contenant le mots "Total"
cumulative_outperformance = cumulative_outperformance.drop(cumulative_outperformance.filter(regex='Total').columns, axis=1)

display(cumulative_outperformance)

,saa minus bmk,manager-saa minus saa
Date,,
2020-03-05,0.000000,0.000000
2020-03-06,-0.000315,-0.000065
2020-03-09,-0.000908,0.003967
2020-03-10,-0.000766,0.001488
2020-03-11,-0.000640,-0.000538
...,...,...
2022-10-25,0.002486,0.016565
2022-10-26,0.001861,0.015087
2022-10-27,0.001399,0.013709


## Visual

In [54]:
fig = px.line(cumulative_outperformance, x=cumulative_outperformance.index, y=cumulative_outperformance.columns,

            title='Cumulative out-/under-performance between each successive pair of portfolios')

fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")
fig.show()

## Export

In [55]:
all_ret_series.to_csv(base_path_out+"Cumulative Outperformance.csv")
fig.write_html(base_path_out+"PCumulative Outperformance.html")